In [2]:
#input: the list of edges
#output: incidence matrix of the hypergraph
def edges_to_matrix(E):
    H = IncidenceStructure(E)
    M = H.incidence_matrix()
    return M

#input: the list of edges of a Veblen hypergraph V
#output: the flattening of V
def flat(V):
    L=[]
    for element in V:
        L.append(tuple(element))
    W=Set(L)
    return sorted(tuple(W))

#in: uniformity k, number of edges d, number of vertices n
#out: the list of Veblen hypergraphs with uniformity k on d edges and \leq n vertices
def Veblen(k,d,n):
    lst=[]
    for element in list(hypergraphs.nauty(d, n, uniform=k, multiple_sets=True)):
        M=edges_to_matrix(element)
        vmod = [x%k for x in M*vector([1]*M.ncols())]
        v = [x for x in M*vector([1]*M.ncols())]
        if sum(vmod) == 0 and (0 not in v):
            lst.append(tuple(sorted(element)))
    return(lst)

In [3]:
#input: the list of edges
#output: incidence matrix of the hypergraph
def edges_to_matrix(E):
    H = IncidenceStructure(E)
    M = H.incidence_matrix()
    return M

#input: the incidence matrix
#output: the list of edges
def matrix_to_edges(M):
    H = IncidenceStructure(M)
    E=[]
    for element in H.blocks():
        E.append(tuple(element))
    return E


# input: hypergraph incidence matrix M, vertices X edges
# output: list of all rootings as a list of nonzero (row,col) pairs of M
def rootings(M):
    R = M.nrows()
    if R == 0 or M.ncols() == 0:
        return [[]]
    rootinglist = []
    for rownum in range(R):
        if M[rownum][0] == 0:
            rownum += 1
        else:
            tempmx = M.submatrix(0,1)
            for colnum in range(1,M.ncols()):
                if M[:,colnum] == M[:,0]:
                    for j in range(rownum):
                        tempmx[j,colnum-1] = 0
            templist = rootings(tempmx)
            rootinglist += [[rownum]+L for L in templist]
    return rootinglist


# input: hypergraph incidence matrix M, vertices X edges
# output: list of all digraphs of Euler rootings
def euler_rootings_digraph(M):
    rootinglist = rootings(M)
    R = M.nrows()
    returnlist = []
    edges=matrix_to_edges(M)
    dit=dict([(a,0) for a in edges])
    for L in rootinglist:
        A = zero_matrix(R)
        c=1
        for rownum in range(R):
            dicct=copy(dit)
            for j in range(len(L)):
                if M[rownum,j] != 0:
                    if rownum != L[j]:
                        A[L[j],rownum] += 1
                    else:
                        dicct[edges[j]]+=1
            c*=multinomial(dicct.values())
        D = DiGraph(A)
        if D.is_eulerian():
#             show(D)
            returnlist.append((D,c))
    return returnlist

#in: Eulerian digraph D
#out: number of spanning rooted subtrees of D
def tau(D):
    L = D.laplacian_matrix()
    L1 = L.submatrix(1,1)
    tau = L1.determinant()
    return tau


#in: distinguishable Eulerian digraph D
#out: number of distinguishable Euler tours of D
def count_euler_tours(D):
    c=tau(D)
    for v in D.vertices():
        c *= factorial(D.out_degree(v)-1)
    return(c)

#input: list of edges of a k-uniform hypergraph
#output: Associated coefficient of H
def as_co(E):
    M=edges_to_matrix(E)
    s=0
    for pair in euler_rootings_digraph(M):
        D=pair[0] 
        c=1
        for v in D.vertices():
            c*=D.out_degree(v)
        s+=pair[1]*tau(D)/c
        # show(D)
    return s

In [4]:
#Written by Dr. Alex Duncan 
#Depends on the invariance of the resultant under “change of base” or ring homomorphisms of the coefficient ring.

import time
import datetime

# This is more direct, but probably slower.
def uniform_hypergraph_charpoly_naive( n, edges):
    d = len(edges[0])
    variable_names = ",".join([f"x{i}" for i in range(n)])
    fermat = " + ".join([f"x{i}^{d}" for i in range(n)])
    edge_mons = []
    for e in edges:
        edge_mons.append( "*".join([f"x{i}" for i in e]) )
    edge_poly = " + ".join(edge_mons)
    t_poly = f"t*({fermat}) - {d}*({edge_poly})"

    macaulay2.eval('loadPackage "Resultants"')
    macaulay2.eval(f'QQ[t][{variable_names}]')
    F = macaulay2(t_poly)
    disc = F.discriminant()
    return disc.sage()

# This computes the polynomial by multiple evaluation
# followed by Lagrange interpolation
# (maybe faster for larger examples?)
# It gives an estimated time remaining counter.
def uniform_hypergraph_charpoly( n, edges, estimate_time = True):
    d = len(edges[0])

    variable_names = ",".join([f"x{i}" for i in range(n)])
    fermat = " + ".join([f"x{i}^{d}" for i in range(n)])
    edge_mons = []
    for e in edges:
        edge_mons.append( "*".join([f"x{i}" for i in e]) )
    edge_poly = " + ".join(edge_mons)

    macaulay2.eval('loadPackage "Resultants"')
    macaulay2.eval(f'QQ[{variable_names}]')

    total_degree = n*(d-1)^(n-1)
    points = []
    start_time = time.time()
    for t in range(total_degree+1):
        t_poly = f"{t}*({fermat}) -  {d}*({edge_poly})"
        F = macaulay2(t_poly)
        disc = F.discriminant()
        points.append([t,disc.sage()])
        if estimate_time: # totally optional
            time_passed = time.time() - start_time
            time_remaining = time_passed*(total_degree-t)/(t+1)
            print(f"Time Remaining: {datetime.timedelta(seconds=time_remaining)}")
    R.<t> = PolynomialRing(QQ)
    return R.lagrange_polynomial(points)